In [20]:
import numpy as np
import pandas as pd
import regex as re
from bs4 import BeautifulSoup
import contractions
import unidecode
import re
import pickle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
import string
from sklearn.utils import shuffle
from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification,RobertaForSequenceClassification,RobertaTokenizer
from transformers import EarlyStoppingCallback
# from gensim.parsing.preprocessing import remove_stopwords
from transformers import XLMRobertaModel, XLMRobertaTokenizer
import os


/home/ablstation2/.pyenv/versions/3.6.5/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
# pip install torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchaudio==0.9.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 2041.3 MB 17 kB/s              
     |████████████████████████████████| 20.6 MB 12.1 MB/s            
     |████████████████████████████████| 1.9 MB 10.7 MB/s            
  Attempting uninstall: torch
    Found existing installation: torch 1.10.2
    Uninstalling torch-1.10.2:
      Successfully uninstalled torch-1.10.2
Note: you may need to restart the kernel to use updated packages.


In [2]:

def removeHtml(sentence):
    soup = BeautifulSoup(sentence, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

def removePuncu(sentence):
    res = re.sub(r'[^\w\s]', '', sentence)
    return res

def removeLink(sentence):
    result = re.sub(r'http\S+', '', sentence)
    return result

def removeWhiteSpaces(sentence):
    sentence = sentence.strip()
    return " ".join(sentence.split())

def expand_contractions(sentence):
    sentence = contractions.fix(sentence)
    return sentence

def removeAccented(sentence):
    sentence = unidecode.unidecode(sentence)
    return sentence

def removeSpec(sentence):
    sentence = re.sub("[^A-Z]", " ", sentence, 0, re.IGNORECASE)
    return sentence

def removeNum(sentence):
    pattern = r'[0-9]'
    sentence = re.sub(pattern, '', sentence)
    return sentence

def lowerCase(sentence):
    sentence = sentence.lower()
    return sentence

# with open(r'C:\Users\LEGION\Documents\Untitled Folder 2\Emoji_Dict.p', 'rb') as fp:
#     Emoji_Dict = pickle.load(fp)
# Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

# def convertEmoji(sentence):
#     for emot in Emoji_Dict:
#         sentence = re.sub(r'('+emot+')', "".join(Emoji_Dict[emot].replace(",","").replace(":","").split()),sentence)
#     return sentence

def add_labels(value):
    if value == 'positive':
        return 2
    elif value == 'negative':
        return 0
    elif value == 'neutral':
        return 1
    

In [3]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred,average='weighted')
    precision = precision_score(y_true=labels, y_pred=pred,average='weighted')
    f1 = f1_score(y_true=labels, y_pred=pred,average='weighted')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


In [4]:
max_length = 128
num_labels = 3

model_name = 'roberta-base'

# safe_model_name = sanitize(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels = num_labels, ignore_mismatched_sizes = True).to("cuda")


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [ ]:
model_results_dict = {"Model_Name":[], "accuracy": [], 'recall': [], 'precision': [], "f1":[]}


In [5]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [16]:
import os

In [7]:
train_data = pd.read_csv('/home/ablstation2/Documents/gaurav/translated_dataset/test/ts_translated_test.tsv', sep = '\t')
val_data = pd.read_csv('/home/ablstation2/Documents/gaurav/translated_dataset/val/ts_translated.tsv', sep = '\t')
train_data['categorical_labels'] = train_data['label'].map(add_labels)
val_data['categorical_labels'] = val_data['label'].map(add_labels)

In [8]:
def preprocessing(data):
    sent_or_word = 'translation_sentence_level'
    data[sent_or_word] = data[sent_or_word].apply(lambda x: lowerCase(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removePuncu(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeSpec(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeHtml(x))

    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeAccented(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeLink(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeNum(x))

    data[sent_or_word] = data[sent_or_word].apply(lambda x: expand_contractions(x))
#     da[sent_or_word] = da[sent_or_word].apply(lambda x: convertEmoji(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeWhiteSpaces(x))
    x = data[sent_or_word].to_list()
    y = list(data['categorical_labels']) 
    
    return x,y


In [9]:
x_train,y_train = preprocessing(train_data)
x_val, y_val = preprocessing(val_data)

x_train_tokenized = tokenizer(x_train, padding=True, truncation=True, max_length=max_length,add_special_tokens=True)
x_val_tokenized = tokenizer(x_val, padding=True, truncation=True, max_length=max_length,add_special_tokens=True)

train_dataset = Dataset(x_train_tokenized, y_train)
val_dataset = Dataset(x_val_tokenized, y_val)



In [10]:
x = 'ts_translated.tsv'

In [14]:
args = TrainingArguments(
    output_dir="/home/ablstation2/Documents/gaurav/translated_dataset/checkpoints/{}_{}".format(model_name,x.split(".")[0]),
    evaluation_strategy="epoch",
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    num_train_epochs=5,
    seed=7,
    load_best_model_at_end=True,
    save_strategy='epoch',
    overwrite_output_dir=True
      #save_strategy = "no"
  )

trainer = Trainer(
      model=model,
      args=args,
      train_dataset=train_dataset,
      eval_dataset=val_dataset,
      compute_metrics=compute_metrics,
      callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
  )


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [14]:

def removeHtml(sentence):
    soup = BeautifulSoup(sentence, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

def removePuncu(sentence):
    res = re.sub(r'[^\w\s]', '', sentence)
    return res

def removeLink(sentence):
    result = re.sub(r'http\S+', '', sentence)
    return result

def removeWhiteSpaces(sentence):
    sentence = sentence.strip()
    return " ".join(sentence.split())

def expand_contractions(sentence):
    sentence = contractions.fix(sentence)
    return sentence

def removeAccented(sentence):
    sentence = unidecode.unidecode(sentence)
    return sentence

def removeSpec(sentence):
    sentence = re.sub("[^A-Z]", " ", sentence, 0, re.IGNORECASE)
    return sentence

def removeNum(sentence):
    pattern = r'[0-9]'
    sentence = re.sub(pattern, '', sentence)
    return sentence
# def preprocessing(data):
#     sent_or_word = 'translation_sentence_level'
#     data[sent_or_word] = data[sent_or_word].apply(lambda x: lowerCase(x))
#     data[sent_or_word] = data[sent_or_word].apply(lambda x: removePuncu(x))
#     data[sent_or_word] = data[sent_or_word].apply(lambda x: removeSpec(x))
#     data[sent_or_word] = data[sent_or_word].apply(lambda x: removeHtml(x))

#     data[sent_or_word] = data[sent_or_word].apply(lambda x: removeAccented(x))
#     data[sent_or_word] = data[sent_or_word].apply(lambda x: removeLink(x))
#     data[sent_or_word] = data[sent_or_word].apply(lambda x: removeNum(x))

#     data[sent_or_word] = data[sent_or_word].apply(lambda x: expand_contractions(x))
# #     da[sent_or_word] = da[sent_or_word].apply(lambda x: convertEmoji(x))
#     data[sent_or_word] = data[sent_or_word].apply(lambda x: removeWhiteSpaces(x))
#     x = data[sent_or_word].to_list()
#     y = list(data['categorical_labels']) 
    
#     return x,y

def lowerCase(sentence):
    sentence = sentence.lower()
    return sentence

def removeUser(sentence):
    remove = re.compile('(\s*)user(\s*)')
    sentence = remove.sub(" ", sentence)
    return sentence

# with open(r'C:\Users\LEGION\Documents\Untitled Folder 2\Emoji_Dict.p', 'rb') as fp:
#     Emoji_Dict = pickle.load(fp)
# Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

# def convertEmoji(sentence):
#     for emot in Emoji_Dict:
#         sentence = re.sub(r'('+emot+')', "".join(Emoji_Dict[emot].replace(",","").replace(":","").split()),sentence)
#     return sentence

def add_labels(value):
    if value == 'positive':
        return 2
    elif value == 'negative':
        return 0
    elif value == 'neutral':
        return 1
    

In [27]:
# def preprocessing(data):
#     sent_or_word = 'translation_sentence_level'
#     data[sent_or_word] = data[sent_or_word].apply(lambda x: lowerCase(x))
#     data[sent_or_word] = data[sent_or_word].apply(lambda x: removePuncu(x))
#     data[sent_or_word] = data[sent_or_word].apply(lambda x: removeSpec(x))
#     data[sent_or_word] = data[sent_or_word].apply(lambda x: removeHtml(x))

#     data[sent_or_word] = data[sent_or_word].apply(lambda x: removeAccented(x))
#     data[sent_or_word] = data[sent_or_word].apply(lambda x: removeLink(x))
#     data[sent_or_word] = data[sent_or_word].apply(lambda x: removeNum(x))

#     data[sent_or_word] = data[sent_or_word].apply(lambda x: expand_contractions(x))
# #     da[sent_or_word] = da[sent_or_word].apply(lambda x: convertEmoji(x))
#     data[sent_or_word] = data[sent_or_word].apply(lambda x: removeWhiteSpaces(x))
#     x = data[sent_or_word].to_list()
#     y = list(data['categorical_labels']) 
    
#     return x,y

def processing(sent):
    removeHtml(sent)
    removePuncu(sent)
    removeLink(sent)
    removeWhiteSpaces(sent)
    expand_contractions(sent)
    removeAccented(sent)
    removeSpec(sent)
    removeNum(sent)
    lowerCase(sent)
    

    


In [42]:
import pandas as pd
data = pd.read_csv('/home/ablstation2/Documents/gaurav/emojiRemovedFiles/test/ig_translated.tsv', sep = '\t')

In [44]:
data

,Unnamed: 0,ID,tweet,label,translation_word_level,translation_sentence_level
0,0,ig_dev_01516,I read kitikpa gba gbuo gi na empathy in my fa...,negative,i read tick run kill you and empathy in my fat...,i read kitikpa go kill you with empathy in my ...
1,1,ig_dev_01400,user user Ọ di egwu 😂,negative,user user He husband fear 😂,user user it's awesome
2,2,ig_dev_00590,user Kedu side ebe o di nwoke oma🙄 Ike umu asa...,neutral,user What side somewhere o husband man ok🙄 Str...,user which side is he a good man the strength ...
3,3,ig_dev_00641,Welcome to KULELEZONE with user amp user NowPl...,neutral,Welcome to KULELEZONE with user amp user NowPl...,welcome to kulelezone with user amp user nowpl...
4,4,ig_dev_01812,Azigbakwa ihe ojoo,negative,Also sent something bad,don't send bad things
...,...,...,...,...,...,...
1836,1836,ig_dev_00906,user Let it be sir Anwụ cha gị eh ịga ekwete n...,neutral,user let it be sir Death casino you eh to go a...,"user let it be sir, don't you want to go to th..."
1837,1837,ig_dev_01792,user Tohwagi onu Ewu,negative,user Tohwagi mouth Goat,user tohwagi's mouth
1838,1838,ig_dev_01097,user Nna adiro m ezo onu agwa mmadu eziokwu,neutral,user Father adiro i hidden mouth Character peo...,user my father hides his mouth and tells peopl...
1839,1839,ig_dev_00236,user Eee nwanyioma arahurum nke oma Udo🙏,positive,user Yes woman I slept which good Peace 🙏,"user yes, my dear, i have been blessed. peace"


In [46]:
data['translation_sentence_level'].isnull() == True

0       False
1       False
2       False
3       False
4       False
        ...  
1836    False
1837    False
1838    False
1839    False
1840    False
Name: translation_sentence_level, Length: 1841, dtype: bool

In [18]:
x = y[0]

In [28]:
processing(x)

In [15]:
trainer.train()
trainer.save_model("/home/ablstation2/Documents/gaurav/translated_dataset/model/{}_{}".format(model_name,x.split(".")[0]))

test_data = pd.read_csv("/home/ablstation2/Documents/gaurav/translated_dataset/test/ts_translated_test.tsv",sep='\t')
test_data['categorical_labels'] = test_data['label'].map(add_labels)

X_test, y_test = preprocessing(test_data)


X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=max_length,add_special_tokens=True)


test_dataset = Dataset(X_test_tokenized, y_test)


predictions = trainer.predict(test_dataset)

pred = np.argmax(predictions.predictions, axis=-1)

  
accuracy = accuracy_score(y_true=y_test, y_pred=pred)
recall = recall_score(y_true=y_test, y_pred=pred,average='weighted')
precision = precision_score(y_true=y_test, y_pred=pred,average='weighted')
f1 = f1_score(y_true=y_test, y_pred=pred,average='weighted')

print(model_name)
print("accuracy: {}".format(accuracy))
print("recall: {}".format(recall))
print("precision: {}".format(precision))
print("f1: {}".format(f1))


/home/ablstation2/.pyenv/versions/3.6.5/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 203
  Num Epochs = 5
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 1
  Total optimization steps = 5
  0%|          | 0/5 [00:00<?, ?it/s]/home/ablstation2/.pyenv/versions/3.6.5/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
 20%|██        | 1/5 [00:00<00:01,  3.54it/s]***** Running Evaluation *****

{'eval_loss': 1.5276384353637695, 'eval_accuracy': 0.4074074074074074, 'eval_precision': 0.6088369070825211, 'eval_recall': 0.4074074074074074, 'eval_f1': 0.2955624622291289, 'eval_runtime': 0.0587, 'eval_samples_per_second': 1380.984, 'eval_steps_per_second': 17.049, 'epoch': 1.0}


Model weights saved in /home/ablstation2/Documents/gaurav/translated_dataset/checkpoints/roberta-base_ts_translated/checkpoint-1/pytorch_model.bin
/home/ablstation2/.pyenv/versions/3.6.5/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
 40%|████      | 2/5 [00:02<00:03,  1.27s/it]***** Running Evaluation *****
  Num examples = 81
  Batch size = 256
/home/ablstation2/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                             
 40%|████      | 2/5 [00:02<00:03,  1.27s/it]Saving model check

{'eval_loss': 1.0464285612106323, 'eval_accuracy': 0.5308641975308642, 'eval_precision': 0.47826363998407, 'eval_recall': 0.5308641975308642, 'eval_f1': 0.48388730610952835, 'eval_runtime': 0.0538, 'eval_samples_per_second': 1505.987, 'eval_steps_per_second': 18.592, 'epoch': 2.0}


Model weights saved in /home/ablstation2/Documents/gaurav/translated_dataset/checkpoints/roberta-base_ts_translated/checkpoint-2/pytorch_model.bin
/home/ablstation2/.pyenv/versions/3.6.5/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
 60%|██████    | 3/5 [00:04<00:03,  1.62s/it]***** Running Evaluation *****
  Num examples = 81
  Batch size = 256
/home/ablstation2/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                             
 60%|██████    | 3/5 [00:04<00:03,  1.62s/it]Saving model check

...

{'eval_loss': 1.1989210844039917, 'eval_accuracy': 0.5679012345679012, 'eval_precision': 0.5342375239088855, 'eval_recall': 0.5679012345679012, 'eval_f1': 0.4782089904896922, 'eval_runtime': 0.078, 'eval_samples_per_second': 1038.058, 'eval_steps_per_second': 12.816, 'epoch': 5.0}


Model weights saved in /home/ablstation2/Documents/gaurav/translated_dataset/checkpoints/roberta-base_ts_translated/checkpoint-5/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /home/ablstation2/Documents/gaurav/translated_dataset/checkpoints/roberta-base_ts_translated/checkpoint-2 (score: 1.0464285612106323).
100%|██████████| 5/5 [00:10<00:00,  2.07s/it]
Saving model checkpoint to /home/ablstation2/Documents/gaurav/translated_dataset/model/roberta-base_ts_translated
Configuration saved in /home/ablstation2/Documents/gaurav/translated_dataset/model/roberta-base_ts_translated/config.json


{'train_runtime': 10.398, 'train_samples_per_second': 97.615, 'train_steps_per_second': 0.481, 'train_loss': 0.824158763885498, 'epoch': 5.0}


Model weights saved in /home/ablstation2/Documents/gaurav/translated_dataset/model/roberta-base_ts_translated/pytorch_model.bin
***** Running Prediction *****
  Num examples = 203
  Batch size = 256
/home/ablstation2/.pyenv/versions/3.6.5/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
  0%|          | 0/1 [00:00<?, ?it/s]

roberta-base
accuracy: 0.7684729064039408
recall: 0.7684729064039408
precision: 0.8218685631534288
f1: 0.7236620872531647
